In [1]:
import glob as glob
import pandas as pd
import requests

# CIViC

In [2]:
handle_name = 'sources/pmid.civic.2021-02-04.txt'
handle = glob.glob(handle_name)

if handle:
    civic_records = pd.read_csv(handle[0], sep='\t')
else:
    request = 'https://civicdb.org/api/sources?count=5000'
    r = requests.get(request)
    civic_records = r.json()['records']
    records = []
    for record in civic_records:
        series = pd.Series(record).loc[['name', 'citation', 'citation_id', 'source_type', 'source_url']]
        records.append(series.to_frame())
    dataframe = pd.concat(records, axis=1, ignore_index=True).T
    civic_records = dataframe[dataframe['source_type'].eq('PubMed')]
    civic_records.to_csv(handle_name, sep='\t', index=False)
civic_pmids = civic_records['citation_id'].drop_duplicates().sort_values().tolist()
print(len(civic_pmids))

2785


In [3]:
for item in ['variants', 'assertions', 'evidence_items']:
    handle_name = f'sources/{item}.civic.2021-02-04.json'
    handle = glob.glob(handle_name)

    request = f'https://civicdb.org/api/{item}?count=10000'
    r = requests.get(request)
    civic_records = r.json()['records']

    import json
    with open(handle_name, 'w') as outfile:
        json.dump(civic_records, outfile)

    print(len(civic_records))

2621
42
7775
